# Defining Custom Display Logic for Your Own Objects

## Overview

In Python, objects can declare their textual representation using the `__repr__` method.  IPython expands on this idea and allows objects to declare other, richer representations including:

* HTML
* JSON
* PNG
* JPEG
* SVG
* LaTeX

This Notebook shows how you can add custom display logic to your own classes, so that they can be displayed using these rich representations. There are two ways of accomplishing this:

1. Implementing special display methods such as `_repr_html_`.
2. Registering a display function for a particular type.

In this Notebook we show how both approaches work.

Before we get started, we will import the various display functions for displaying the different formats we will create.

In [ ]:
from IPython.display import display
from IPython.display import (
    display_html, display_jpeg, display_png,
    display_javascript, display_svg, display_latex
)

## Implementing special display methods

The main idea of the first approach is that you have to implement special display methods, one for each representation you want to use. Here is a list of the names of the special methods and the values they must return:

* `_repr_html_`: return raw HTML as a string
* `_repr_json_`: return raw JSON as a string
* `_repr_jpeg_`: return raw JPEG data
* `_repr_png_`: return raw PNG data
* `_repr_svg_`: return raw SVG data as a string
* `_repr_latex_`: return LaTeX commands in a string surrounded by "$".

### Model Citizen: pandas

A prominent example of a package that has IPython-aware rich representations of its objects is [pandas](http://pandas.pydata.org/).

A pandas DataFrame has a rich HTML table representation,
using `_repr_html_`.


In [ ]:
import pandas

In [ ]:
%%writefile data.csv
Date,Open,High,Low,Close,Volume,Adj Close
2012-06-01,569.16,590.00,548.50,584.00,14077000,581.50
2012-05-01,584.90,596.76,522.18,577.73,18827900,575.26
2012-04-02,601.83,644.00,555.00,583.98,28759100,581.48
2012-03-01,548.17,621.45,516.22,599.55,26486000,596.99
2012-02-01,458.41,547.61,453.98,542.44,22001000,540.12
2012-01-03,409.40,458.24,409.00,456.48,12949100,454.53


In [ ]:
df = pandas.read_csv("data.csv")
pandas.set_option('display.notebook_repr_html', False)
df

rich HTML can be activated via `pandas.set_option`.

In [ ]:
pandas.set_option('display.notebook_repr_html', True)
df

In [ ]:
lines = df._repr_html_().splitlines()
print("\n".join(lines[:20]))

### Exercise

Write a simple `Circle` Python class.  Don't even worry about properties such as radius, position, colors, etc. To help you out use the following representations (remember to wrap them in Python strings):

For HTML:

    &#x25CB;

For SVG:

    <svg width="100px" height="100px">
        <circle cx="50" cy="50" r="20" stroke="black" stroke-width="1" fill="white"/>
    </svg>

For LaTeX (wrap with `$` and use a raw Python string):

    \bigcirc

After you write the class, create an instance and then use `display_html`, `display_svg` and `display_latex` to display those representations.

Tips : you can slightly tweek the representation to know from which `_repr_*_` method it came from. 
For example in my solution the svg representation is blue, and the HTML one show "`HTML`" between brackets.

### Solution

Here is my simple `Circle` class:

In [ ]:
%load mycircle.py

Now create an instance and use the display methods:

In [ ]:
c = MyCircle()

In [ ]:
display_html(c)

In [ ]:
display_svg(c)

In [ ]:
display_latex(c)

In [ ]:
display_javascript(c)

## Adding IPython display support to existing objects

When you are directly writing your own classes, you can adapt them for display in IPython by following the above example.  But in practice, we often need to work with existing code we can't modify.  We now illustrate how to add these kinds of extended display capabilities to existing objects. To continue with our example above, we will add a PNG representation to our `Circle` class using Matplotlib.

### Model citizen: sympy

[SymPy](http://sympy.org) is another model citizen that defines rich representations of its object.
Unlike pandas above, sympy registers display formatters via IPython's display formatter API, rather than declaring `_repr_mime_` methods.

In [ ]:
from sympy import Rational, pi, exp, I, symbols
x, y, z = symbols("x y z")

In [ ]:
r = Rational(3,2)*pi + exp(I*x) / (x**2 + y)
r

SymPy provides an `init_printing` function that sets up advanced formatting (including $\LaTeX$
representations) of its objects:

In [ ]:
from sympy.interactive.printing import init_printing
init_printing(use_latex=False)  # no LaTeX enabled, we get unicode printing
r

In [ ]:
# Now activate LaTeX
init_printing( use_latex=True )
r

To add a display method to an existing class, we must use IPython's display formatter API.  Here we show all of the available formatters:

In [ ]:
ip = get_ipython()
ip.display_formatter.formatters

Let's grab the PNG formatter:

In [ ]:
png_f = ip.display_formatter.formatters['image/png']

We will use the `for_type` method to register our display function.

In [ ]:
png_f.for_type?

As the docstring describes, we need to define a function the takes the object as a parameter and returns the raw PNG data.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
class AnotherCircle(object):    
    def __repr__(self):
        return "<r Circle with r=1 at (0,0)>"
    
c = AnotherCircle()
c

In [ ]:
from IPython.core.pylabtools import print_figure

def png_circle(circle):
    """Render AnotherCircle to png data using matplotlib"""
    fig, ax = plt.subplots()
    patch = plt.Circle((0,0), radius=1.0, fc='r')
    ax.add_patch(patch)
    plt.axis('scaled')
    data = print_figure(fig, 'png')
    # We MUST close the figure, otherwise IPython's display machinery
    # will pick it up and send it as output, resulting in a double display
    plt.close(fig)
    return data

Now we register the display function for the type:

In [ ]:
png_f.for_type(AnotherCircle, png_circle)

Now all `Circle` instances have PNG representations!

In [ ]:
c2 = AnotherCircle()
c2

In [ ]:
display_png(c2)

### Exercise

Register for the `AnotherCircle` class a LaTeX formatter that uses the same value as the one in the `Circle` class above, but without adding new methods to `AnotherCircle`. That is, use the `text/latex` formatter similarly to how we used the `image/png` one.

### Solution

In [ ]:
%load mycircle2.py